<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/bilanco_yfinans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyelim
!pip install investpy pandas matplotlib qgrid

import investpy
import yfinance as yf
import pandas as pd
import qgrid
from IPython.display import display
from requests.exceptions import HTTPError

# Investpy kullanarak Türkiye'deki tüm hisseleri çek
turkish_stocks = investpy.stocks.get_stocks(country='Turkey')
hisseler = turkish_stocks['symbol'].tolist()

# Yahoo Finance'e uygun hale getirme (".IS" ekleme)
hisse_listesi = [hisse + ".IS" for hisse in hisseler]

# Verileri toplamak için bir liste oluştur
veriler = []
hatali_hisseler = []


# FD/FAVÖK hesaplama fonksiyonu
def hesapla_fd_favok(piyasa_degeri, toplam_borc, nakit, favok):
    if piyasa_degeri and favok:  # Piyasa Değeri ve FAVÖK varsa
        firma_degeri = piyasa_degeri + (toplam_borc or 0) - (nakit or 0)
        return firma_degeri / favok if favok != 0 else None
    return None

# PD/DD hesaplama fonksiyonu
def hesapla_pd_dd(piyasa_degeri, defter_degeri):
    if defter_degeri != 0:  # Defter Değeri sıfır değilse
        return piyasa_degeri / defter_degeri
    return None

# Verileri toplama ve PD/DD hesaplaması
for hisse in hisse_listesi:
    try:

        # Yahoo Finance'den finansal verileri çek
        ticker = yf.Ticker(hisse)
        finansal_veri = ticker.info

        # Eğer finansal veri boşsa hisseyi es geç
        if not finansal_veri or "quoteType" not in finansal_veri:
            print(f"{hisse} için ismi değişmiş. Es geçiliyor.")
            hatali_hisseler.append(hisse)
            continue

        # Sektör N/A ise, bu veriyi atla
        if finansal_veri.get("sector", "N/A") == "N/A":
            print(f"{hisse} için sektör bilgisi mevcut değil. Es geçiliyor.")
            hatali_hisseler.append(hisse)
            continue
        # Finansal metrikler
        piyasa_degeri = finansal_veri.get("marketCap", 0)
        firma_degeri = finansal_veri.get("enterpriseValue", 0)
        ebitda = finansal_veri.get("ebitda", 0)
        toplam_borc = finansal_veri.get("totalDebt", 0)
        nakit = finansal_veri.get("totalCash", 0)
        defter_degeri = finansal_veri.get("bookValue", 0)
        dolaşımdaki_hisse_adedi = finansal_veri.get("sharesOutstanding", 0)
        brüt_kar_marji = finansal_veri.get("grossMargins", 0)
        faaliyet_kar_marji = finansal_veri.get("operatingMargins", 0)
        varlik_karliligi = finansal_veri.get("returnOnAssets", 0)
        ozsermaye_karliligi = finansal_veri.get("returnOnEquity", 0)
        net_kar = finansal_veri.get("netIncomeToCommon", 0)
        toplam_gelir = finansal_veri.get("totalRevenue", 0)
        hisse_basi_gelir = finansal_veri.get("revenuePerShare", 0)
        temettü_getirisi = finansal_veri.get("dividendYield", 0)
        temettü_orani = finansal_veri.get("dividendRate", 0)
        fk_trailing = finansal_veri.get("trailingPE", 0)
        fk_forward = finansal_veri.get("forwardPE", 0)
        pd_dd = hesapla_pd_dd(piyasa_degeri, defter_degeri)
        peg_orani = finansal_veri.get("pegRatio", 0)
        gelir_buyumesi = finansal_veri.get("revenueGrowth", 0)
        kar_buyumesi = finansal_veri.get("earningsGrowth", 0)
        ceyreklik_kar_buyumesi = finansal_veri.get("earningsQuarterlyGrowth", 0)
        brut_kar = finansal_veri.get("grossProfits", 0)
        hizli_oran = finansal_veri.get("quickRatio", 0)
        borc_ozkaynak_orani = finansal_veri.get("debtToEquity", 0)
        cari_oran = finansal_veri.get("currentRatio", 0)

        # FD/FAVÖK hesaplaması
        fd_favok = hesapla_fd_favok(piyasa_degeri, toplam_borc, nakit, ebitda)

        # Net Kar Marjı (%) hesapla (Net Kar / Toplam Gelir)
        if toplam_gelir != 0:
            net_kar_marji = (net_kar / toplam_gelir) * 100
        else:
            net_kar_marji = 0

        # Verileri listeye ekleme
        veriler.append({
            "Hisse": hisse,
            "Sektör": finansal_veri.get("sector", "N/A"),
            "Endüstri": finansal_veri.get("industry", "N/A"),
            "Piyasa Değeri (₺)": piyasa_degeri,
            "Firma Değeri (₺)": firma_degeri,
            "FAVÖK (₺)": ebitda,
            "FD/FAVÖK": fd_favok,
            "PD/DD": pd_dd,  # PD/DD oranı
            "Dolaşımdaki Hisse Adedi": dolaşımdaki_hisse_adedi,
            "Brüt Kar Marjı (%)": brüt_kar_marji,
            "Faaliyet Kar Marjı (%)": faaliyet_kar_marji,
            "ROA (Varlık Karlılığı) (%)": varlik_karliligi,
            "ROE (Özsermaye Karlılığı) (%)": ozsermaye_karliligi,
            "Net Kar (₺)": net_kar,
            "Net Kar Marjı (%)": net_kar_marji,
            "Toplam Gelir (₺)": toplam_gelir,
            "Hisse Başına Gelir (₺)": hisse_basi_gelir,
            "Temettü Getirisi (%)": temettü_getirisi,
            "Temettü Oranı (%)": temettü_orani,
            "F/K (Trailing)": fk_trailing,
            "F/K (Forward)": fk_forward,
            "PEG Oranı": peg_orani,
            "Gelir Büyümesi (%)": gelir_buyumesi,
            "Kâr Büyümesi (%)": kar_buyumesi,
            "Çeyreklik Kâr Büyümesi (%)": ceyreklik_kar_buyumesi,
            "Brüt Kâr (₺)": brut_kar,
            "Hızlı Oran": hizli_oran,
            "Borç/Özkaynak Oranı": borc_ozkaynak_orani,
            "Cari Oran": cari_oran,
        })
    except HTTPError as http_err:
        if http_err.response.status_code == 404:
            print(f"{hisse} için veri bulunamadı (404). Es geçiliyor.")
            hatali_hisseler.append(hisse)
        else:
            print(f"{hisse} için farklı bir hata oluştu: {http_err}")
            hatali_hisseler.append(hisse)
    except Exception as e:
        print(f"{hisse} için bir hata oluştu: {e}")
        hatali_hisseler.append(hisse)

# Verileri bir DataFrame'e dönüştür
df = pd.DataFrame(veriler)

# Verileri sıralama (en iyi finansal verilerden en kötüye doğru)
df = df.sort_values(by=["ROE (Özsermaye Karlılığı) (%)", "Net Kar Marjı (%)", "PD/DD"], ascending=[False, False, True])

# Sayıları yuvarlama ve formatlama
df["Piyasa Değeri (₺)"] = df["Piyasa Değeri (₺)"].apply(lambda x: round(x / 1e9, 2) if x != 0 else 0)  # Milyar olarak yazma
df["PD/DD"] = df["PD/DD"].round(2)
df["FAVÖK (₺)"] = df["FAVÖK (₺)"].apply(lambda x: round(x / 1e9, 2) if x != 0 else 0)  # Milyar olarak yazma
df["FD/FAVÖK"] = df["FD/FAVÖK"].round(2)
df["Net Kar Marjı (%)"] = df["Net Kar Marjı (%)"].round(2)
df["Brüt Kar Marjı (%)"] = df["Brüt Kar Marjı (%)"].round(2)
df["Faaliyet Kar Marjı (%)"] = df["Faaliyet Kar Marjı (%)"].round(2)
df["ROA (Varlık Karlılığı) (%)"] = df["ROA (Varlık Karlılığı) (%)"].round(2)
df["ROE (Özsermaye Karlılığı) (%)"] = df["ROE (Özsermaye Karlılığı) (%)"].round(2)
df["Temettü Getirisi (%)"] = df["Temettü Getirisi (%)"].round(2)
df["F/K (Trailing)"] = df["F/K (Trailing)"].round(2)
df["F/K (Forward)"] = df["F/K (Forward)"].round(2)
df["Gelir Büyümesi (%)"] = df["Gelir Büyümesi (%)"].round(2)
df["Kâr Büyümesi (%)"] = df["Kâr Büyümesi (%)"].round(2)
df["Çeyreklik Kâr Büyümesi (%)"] = df["Çeyreklik Kâr Büyümesi (%)"].round(2)
df["Hızlı Oran"] = df["Hızlı Oran"].round(2)
df["Borç/Özkaynak Oranı"] = df["Borç/Özkaynak Oranı"].round(2)
df["Cari Oran"] = df["Cari Oran"].round(2)

# Veriyi ekrana göster
display(df)

# Verileri Excel'e kaydet
excel_dosya = "BIST_Tum_Hisseler_Finansal_Veriler_Turkce.xlsx"
df.to_excel(excel_dosya, index=False, float_format="%.2f")
print(f"\nVeriler başarıyla '{excel_dosya}' dosyasına kaydedildi!")



ADANA.IS için sektör bilgisi mevcut değil. Es geçiliyor.
ADBGR.IS için sektör bilgisi mevcut değil. Es geçiliyor.
ADNAC.IS için sektör bilgisi mevcut değil. Es geçiliyor.
BOYP.IS için sektör bilgisi mevcut değil. Es geçiliyor.
ALYAG.IS için sektör bilgisi mevcut değil. Es geçiliyor.
ANACM.IS için sektör bilgisi mevcut değil. Es geçiliyor.
ARMDA.IS için sektör bilgisi mevcut değil. Es geçiliyor.
ASLAN.IS için sektör bilgisi mevcut değil. Es geçiliyor.
BOLUC.IS için sektör bilgisi mevcut değil. Es geçiliyor.
DENCM.IS için sektör bilgisi mevcut değil. Es geçiliyor.
DENIZ.IS için sektör bilgisi mevcut değil. Es geçiliyor.


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EGCYH.IS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EGCYH.IS&crumb=AmdJDgEAN0Y


EGCYH.IS için ismi değişmiş. Es geçiliyor.


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EGCYO.IS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EGCYO.IS&crumb=AmdJDgEAN0Y


EGCYO.IS için ismi değişmiş. Es geçiliyor.
GYHOL.IS için sektör bilgisi mevcut değil. Es geçiliyor.
GUSGR.IS için sektör bilgisi mevcut değil. Es geçiliyor.
ISGSY.IS için sektör bilgisi mevcut değil. Es geçiliyor.


...

SALIX.IS için ismi değişmiş. Es geçiliyor.


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PMVR3.IS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PMVR3.IS&crumb=AmdJDgEAN0Y


PMVR3.IS için ismi değişmiş. Es geçiliyor.
DERAS.IS için sektör bilgisi mevcut değil. Es geçiliyor.


,Hisse,Sektör,Endüstri,Piyasa Değeri (₺),Firma Değeri (₺),FAVÖK (₺),FD/FAVÖK,PD/DD,Dolaşımdaki Hisse Adedi,Brüt Kar Marjı (%),...,F/K (Trailing),F/K (Forward),PEG Oranı,Gelir Büyümesi (%),Kâr Büyümesi (%),Çeyreklik Kâr Büyümesi (%),Brüt Kâr (₺),Hızlı Oran,Borç/Özkaynak Oranı,Cari Oran
107,ESCOM.IS,Technology,Information Technology Services,2.70,2951204096,-0.01,-286.86,2.179101e+08,49992100,1.00,...,8.390358,0.00,0,-0.82,-0.78,-0.78,0,0.52,0.00,0.61
41,BFREN.IS,Consumer Cyclical,Auto Parts,22.84,23572457472,0.07,345.07,5.704211e+09,30636700,0.05,...,125.929054,0.00,0,0.21,0.00,0.00,0,1.32,175.73,1.75
215,RYSAS.IS,Industrials,Integrated Freight & Logistics,36.58,39988928512,5.00,7.75,1.371579e+10,2000000000,0.50,...,25.402779,0.00,0,1.80,3.45,7.90,0,1.55,104.63,1.66
272,DENGE.IS,Financial Services,Asset Management,0.85,1144372736,0.20,5.28,2.503672e+08,298942016,0.00,...,0.943522,0.00,0,0.43,0.00,0.00,0,0.92,15.37,1.08
214,RYGYO.IS,Real Estate,REIT - Industrial,27.82,28788119552,3.56,8.56,1.299453e+09,2000000000,0.89,...,1.540421,0.00,0,2.33,6.09,6.09,0,0.84,9.01,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,CRFSA.IS,Consumer Defensive,Grocery Stores,13.02,18247278592,0.23,82.48,3.253985e+08,127774000,0.23,...,0,0.00,0,-0.00,0.00,0.00,0,0.25,166.51,0.65
79,DITAS.IS,Consumer Cyclical,Auto Parts,1.27,1836444928,-0.18,-11.40,3.246362e+08,85000000,0.05,...,0,0.00,0,0.13,0.00,0.00,0,0.34,237.90,0.74
44,BIZIM.IS,Consumer Cyclical,Department Stores,2.21,1334883072,-0.72,-4.00,3.432778e+07,80476096,0.14,...,0,-3.61,0,0.41,0.00,0.00,0,0.30,85.24,0.69
271,DAGHL.IS,Financial Services,Asset Management,0.51,491811360,0.01,44.59,5.238438e+08,30000000,0.05,...,0,0.00,0,-0.24,0.00,0.00,0,0.02,8.02,1.02



Veriler başarıyla 'BIST_Tum_Hisseler_Finansal_Veriler_Turkce.xlsx' dosyasına kaydedildi!
